In [44]:
import numpy as np
from numpy.linalg import eig
from math import sqrt

import numpy as np

def ComputeSVD(A):
    """ 
    Compute SVD decomposition of matrix A. Returns tuple (U, Sigma, V) of np.matrix objects.
    """
    ATA = A.T @ A
    eigen_values_V, V = np.linalg.eig(ATA)

    # Sort eigenvalues and eigenvectors in descending order
    idx = np.argsort(eigen_values_V)[::-1]
    eigen_values_V = eigen_values_V[idx]
    V = V[:, idx]

    Sigma = np.diag(np.sqrt(eigen_values_V))

    # Compute U using A, V and Sigma
    U = A @ V @ np.linalg.inv(Sigma)

    return U, Sigma, V

def PseudoInverse(A):
    """ 
    Compute the pseudo inverse of matrix A using its SVD.
    """
    U, Sigma, V = ComputeSVD(A)
    diagvalues = np.diag(Sigma)
    diagvalues = diagvalues**-1
    Sigma_plus = np.diag(diagvalues)

    for i in range(min(A.shape[0], A.shape[1])):
        if Sigma[i, i] != 0:
            Sigma_plus[i, i] = 1 / Sigma[i, i]

    print(V.shape)
    print(Sigma_plus.shape)
    print(U.H.shape)

    A_plus = V @ Sigma_plus @ U.H
    return A_plus


def LinearSolve(A, b):
    """ 
    Given a matrix A and a vector b this function solves the linear
    equations A*x=b by solving the least squares problem of minimizing
    |A*x-b| and returns the optimal x.
    """
    A_plus = PseudoInverse(A)
    x = A_plus @ b
    #x = 0
    return x


In [45]:
# Try the SVD decomposition
A = np.matrix([[1.0, 1.0, 1.0], [1.0, 2.0, 3.0], [1.0, 4.0, 9.0], [1.0, 8.0, 27.0]])
U, Sigma, V = ComputeSVD(A)
print("U:")
print(U)
print("Sigma:")
print(Sigma)
print("V:")
print(V)
print()
print("If the following numbers are nearly zero, SVD seems to be working.")
print(np.linalg.norm(U * Sigma * V.H - A))
print(np.linalg.norm(U.H * U - np.eye(3)))
print(np.linalg.norm(V.H * V - np.eye(3)))
print()

# Try solving a least squares system
b = np.matrix([1.0, 2.0, 3.0, 4.0]).T
x = LinearSolve(A, b)

difference = np.linalg.norm(x - np.linalg.lstsq(A, b, rcond=None)[0])
print("If the following number is nearly zero, " "linear solving seems to be working:", difference)
if np.isclose(difference, 0.0):
    print("✔️ The solution is close to the numpy least squares result.")
else:
    print("❌ The solution seems to be wrong.")

U:
[[-0.04328203 -0.50635895 -0.74576154]
 [-0.1166556  -0.6007988  -0.0414841 ]
 [-0.32673486 -0.546948    0.63915977]
 [-0.93688978  0.28894516 -0.18328554]]
Sigma:
[[30.0685691   0.          0.        ]
 [ 0.          2.16759711  0.        ]
 [ 0.          0.          0.42740494]]
V:
[[-0.04734386 -0.62980366 -0.7753102 ]
 [-0.30193159 -0.73084951  0.61212442]
 [-0.95215328  0.26307098 -0.15555638]]

If the following numbers are nearly zero, SVD seems to be working.
2.100669880799093e-14
2.0906809392281362e-14
5.4541403277016055e-15

(3, 3)
(3, 3)
(3, 4)
If the following number is nearly zero, linear solving seems to be working: 1.172633466287674e-13
✔️ The solution is close to the numpy least squares result.
